<a href="https://colab.research.google.com/github/AjayBora002/movie_recommendation/blob/main/Copy_of_movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.Import Modules


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk

In [ ]:
movies=pd.read_csv("movies.csv")

In [ ]:


def clean_title(title):
    return re.sub(r'[^a-zA-Z0-9 ]', '', title.lower()).strip()


In [ ]:
movies["clean_title"]=movies["title"].apply(clean_title)# this will access a new clean title column


2. FINDING SIMILARITIES BETWEEN THE WORDS AND WITH THE HELP OF IT GIVING THE SIMILAR TYPE OF MOVIE TO THE USER

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer# converting text data into numerical data and finding similarities , this will do it
vectorizer = TfidfVectorizer(ngram_range=(1,2))# this ngram thing will help in finding instead of single single word it will match two consecutive words like toy story together
tfidf = vectorizer.fit_transform(movies["clean_title"])

3.finding similarity and creating search function

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)  # clean the input title
    query_vec = vectorizer.transform([title])  # vectorize it
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]
    return results


4.CREATING AN INTERACTIVE BOX

In [ ]:
import ipywidgets as widgets
from IPython.display import display  # function used to show diiff things as output on notebook


movie_input = widgets.Text(    # creating a widget
    value='Toy Story',
    description = "Movie Title :",  # movie titled will be entered here
    disabled=False   # it will be enabled
)


# this whole thing will create a box to enter movie title , but wont show anything as output
  #NOW MAKING AN OUTPUT WIDGET
movie_list = widgets.Output()

def on_type(data):     # this fun will be called whenever we type something in the box
  with movie_list:
    movie_list.clear_output()

    title=data["new"]
    if len(title)>5:
      display(search(title))

movie_input.observe(on_type, names='value')  # there are diff events of the widgets whenever we input a movie name it is going to call on type and gives value event
display(movie_input, movie_list)




Text(value='Toy Story', description='Movie Title :')

Output()

In [ ]:
ratings=pd.read_csv("ratings.csv")

In [ ]:
ratings.dtypes

,0
userId,int64
movieId,int64
rating,float64
timestamp,int64


5.FINDING SIMILAR USER RECOMMENDATION

In [ ]:
movie_id=1

In [ ]:
similar_users = ratings[(ratings["movieId"] == movie_id)# this is for finding anyone who watched same movie
& (ratings["rating"] > 4)]["userId"].unique()# this is for anyone who watched the movie and gave the rating 5

similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
# this will show the movies that are rated by similar users and find their used ids

In [ ]:
similar_user_recs

,movieId
5101,1
5105,34
5111,110
5114,150
5127,260
...,...
24998854,60069
24998861,67997
24998876,78499
24998884,81591


In [ ]:
similar_user_recs=similar_user_recs.value_counts()/len(similar_users) # this counts the no of 5 rates is given to a movie and then div by len shows recommending percent like top one has 100%

# we are doing this to find per of similar people who watched the movie
similar_user_recs = similar_user_recs[similar_user_recs > .10]  # shows movies having per greater than 10 %

In [ ]:
similar_user_recs

,count
movieId,
1,1.000000
318,0.445607
260,0.403770
356,0.370215
296,0.367295
...,...
953,0.103053
551,0.101195
1222,0.100876


6.now problem is that most of the people like movies like toy story and avengers , what we need is how much movies are liked by users are we also like this will give more accuracy on recommendation

In [ ]:
all_users=ratings[(ratings)["movieId"].isin(similar_user_recs.index) & (ratings["rating"]>4)]

In [ ]:
all_users_recs=all_users["movieId"].value_counts()/len(all_users["userId"].unique())# this will give us percentage of all the users who liked the movies

In [ ]:
rec_percentages=pd.concat([similar_user_recs,all_users_recs],axis=1)# this concatenates the data of how much similar people to us liked and how much avg person liked it
rec_percentages.columns=["similar","all"]

In [ ]:
rec_percentages

,similar,all
movieId,,
1,1.000000,0.124728
318,0.445607,0.342220
260,0.403770,0.222207
356,0.370215,0.235266
296,0.367295,0.284674
...,...,...
953,0.103053,0.045792
551,0.101195,0.040918
1222,0.100876,0.066877


7.NOW WE WANT MOVIES THAT HAS A BIG DIFF BET THESE TWO COLUMN

In [ ]:
rec_percentages["score"]=rec_percentages["similar"]/rec_percentages["all"]
rec_percentages=rec_percentages.sort_values("score",ascending=False)   # using pandas sort method to make an order of the score the higher the score more good will be the recommendation
rec_percentages

,similar,all,score
movieId,,,
1,1.000000,0.124728,8.017414
3114,0.280648,0.053706,5.225654
2355,0.110539,0.025091,4.405452
78499,0.152960,0.035131,4.354038
4886,0.235147,0.070811,3.320783
...,...,...,...
2858,0.216724,0.167634,1.292845
296,0.367295,0.284674,1.290232
79132,0.166817,0.131384,1.269693


8.NOW TAKING TOP 10 RATINGS AND MERGE IT WITH MOVIES DATA

In [ ]:
rec_percentages.head(10).merge(movies,left_index=True,right_on="movieId")

,similar,all,score,movieId,title,genres,clean_title
0,1.000000,0.124728,8.017414,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995
3021,0.280648,0.053706,5.225654,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,toy story 2 1999
2264,0.110539,0.025091,4.405452,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,bugs life a 1998
14813,0.152960,0.035131,4.354038,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,toy story 3 2010
4780,0.235147,0.070811,3.320783,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,monsters inc 2001
580,0.216618,0.067513,3.208539,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,aladdin 1992
6258,0.228139,0.072268,3.156862,6377,Finding Nemo (2003),Adventure|Animation|Children|Comedy,finding nemo 2003
587,0.179400,0.059977,2.991150,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,beauty and the beast 1991
8246,0.203504,0.068453,2.972889,8961,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,incredibles the 2004
359,0.253411,0.085764,2.954762,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,lion king the 1994


9.PUTTING ALL OF THIS INTO A FUNCTION

In [ ]:
def find_similar_movies(movie_id):

    similar_users = ratings[(ratings["movieId"] == movie_id)
    & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) # this is finding recommendation similar to us
    & (ratings["rating"] > 4)]["movieId"]

    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)# adjusting the per of recommendation
    similar_user_recs = similar_user_recs[similar_user_recs > .10]

    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]# this is finding common recommendation among all of the users
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)# concatinating
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]# generating score

    rec_percentages=rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]  #sorting and returning our merge data

10.CREATING AN INTERACTIVE RECOMMENDATION WIDGET

In [ ]:
movie_name_input = widgets.Text(    # creating a widget
    value='Toy Story',
    description = "Movie Title :",  # movie titled will be entered here
    disabled=False   # it will be enabled
)

recommendation_list=widgets.Output()

def on_type(data):
  with recommendation_list:
    recommendation_list.clear_output()# removes old output
    title=data["new"]
    if len(title)>5:
      results=search(title)
      movie_id=results.iloc[0]["movieId"]
      display(find_similar_movies(movie_id))

movie_name_input.observe(on_type,names="value")
display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title :')

Output()